<a href="https://colab.research.google.com/github/Hugo-Zh0/YoloV12-Object-Detection-Project/blob/main/YOLOv12_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🚀 YOLOv12 Object Detection Project

[![Python](https://img.shields.io/badge/Python-3.10%2B-blue.svg)](https://www.python.org/downloads/)
[![Anaconda](https://img.shields.io/badge/Anaconda-Navigator-green.svg)](https://www.anaconda.com/download)
[![VS Code](https://img.shields.io/badge/Editor-VS%20Code-blue.svg)](https://code.visualstudio.com/)
[![Ultralytics](https://img.shields.io/badge/YOLOv12-Ultralytics-yellow.svg)](https://github.com/ultralytics/ultralytics)
[![License](https://img.shields.io/badge/License-MIT-lightgrey.svg)](LICENSE)

---
<br>

## 📌 Overview
A **collaborative group project** by Swinburne University students in partnership with **CSIRO**.  
This repository contains the setup, configuration, and workflow for training and running **YOLOv12** object detection models.

**👨‍💻 Team Members:** Harron, Feng, Bunmi, Huss, Hugo.

---
**Repo:** `Hugo-Zh0/YoloV12-Object-Detection-Project`  
**What you’ll do:**
1. Check runtime & GPU
2. Install dependencies
3. Clone your repo
4. Set paths in repository
5. Train
6. Validate
7. Predict
8. Export
10. Troubleshoot

<br>

#### **Full Process Run Through**

Process 1: Train > Validate > Inference > (repeat steps) to get multiple models with different results

Processs 2: Run Final Model > Gets Final Model > Run Full Test

Process 3: Final Model > Export Model (for deployment)

<br>

#### **What to do after completion or if you don't want to run anymore**

After completing this colab you will need to export the folders(step 9) which includes the repository and runs, as the runtime session will expire when you close the website **(meaning the folders gets deleted)**.

<br>

#### **Starting from previous session**

If you are rerunning this agin, you will need to manually upload the folders back into the google colab again (it has to be zipped up first to be uploaded)
Then you will need to run script to extract the folders back to original state.

From there you can start from Step 1,2,5,6,7,8 (excludes 3-4 as no need to clone repository and setting paths again)

## **Step 1 — 🚀 Runtime & GPU check**

## **Prerequistes**
*   Change runtime type to T4-GPU
*   Change runtime to Python 3
*   Have your dataset already downloaded







In [ ]:
#@title Check Python, CUDA, and PyTorch (Checks if runtime is all correct)

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except Exception as e:
    print("PyTorch not installed yet (will install in next step).")


## Step 2 — ⬇️ Install dependencies

These are the python libraries and Ultralytics libraries needed to run the framework

In [ ]:
#@title Install Ultralytics & helpers

!pip install ultralytics

import torch, cv2, ultralytics
print("Ultralytics:", ultralytics.__version__)
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("OpenCV:", cv2.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.3.186
Torch: 2.8.0+cu126 | CUDA: True
OpenCV: 4.12.0


## Step 3 —  🤖 Clone your repository

Grabs our repository which contains our folder structure/files and folders to get started with our training

In [ ]:
#@title Clone repository from github

REPO_URL = "https://github.com/Hugo-Zh0/YoloV12-Object-Detection-Project"
REPO_DIR = "/content/YoloV12-Object-Detection-Project"

import shutil, os, pathlib
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone -q {REPO_URL} {REPO_DIR}
print("Cloned into:", REPO_DIR)


# 3.1 - Creating folders
Next we will need to manually create two folders under our cloned repository
* datasets
* yaml

Datasets:
* Folder will store our annotated datasets from roboflow

Yaml:

* Folder stores our yaml file from our dataset




In [ ]:
#@title Import Datset Zip File into Colab
from google.colab import files
import zipfile

uploaded = files.upload()

In [ ]:
#@title Unzip Dataset and save to dataset folder

# for koala (comment if using kangaroo)
!unzip -q /content/koala.zip -d /content/YoloV12-Object-Detection-Project/datasets/koala

# for kangaroo (uncomment if using kangaroo)
#!unzip -q /content/kangaroo.zip -d /content/YoloV12-Object-Detection-Project/datasets/kangaroo

## Step 4 — 🎞️ Set model & data paths

**Step 4.1**

Manually move YAML File stored in extracted dataset folder to previously created YAML Folder.

**Step 4.2**

Update the YAML File with proper location paths for: train, val, test

Double click the yaml file and it will open on the side.

**Koala:**

* /content/YoloV12-Object-Detection-Project/datasets/koala/train/images
* /content/YoloV12-Object-Detection-Project/datasets/koala/valid/images
* /content/YoloV12-Object-Detection-Project/datasets/koala/test/images


**Kangaroo:**

* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/train/images
* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/valid/images
* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/test/images

<br>

Finally save the file doing Ctrl+S

## Step 5 — 💥 Train (set your parameters)

This script will train your dataset and store them in the respoitory locations.
You can also set the configs you want to train your dataset, tune it however you like to get the best performance.

In [ ]:
#@title Set Configs and train datasaet

from ultralytics import YOLO
import torch, os, sys, requests

# Direct paths to locations
MODELS_DIR  = "/content/YoloV12-Object-Detection-Project/models"
MODEL_PATH  = os.path.join(MODELS_DIR, "yolo12s.pt")
DATA_YAML   = "/content/YoloV12-Object-Detection-Project/yaml/data.yaml"
RUNS_DIR    = "/content/YoloV12-Object-Detection-Project/runs/completed-training"

print("Model path:", MODEL_PATH)
print("Data yaml :", DATA_YAML)

# Device selection
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load your local weights explicitly
model = YOLO(MODEL_PATH)

# Training configs (leave imgsz as 640) everything else change change
results = model.train(
    data=DATA_YAML, # refers to our dataset yaml file
    epochs=150,
    imgsz=640,
    batch=-1,
    workers=2,
    device=device,
    pretrained=False,
    amp=False,
    project=RUNS_DIR,
    name="train"
)

print("Save dir:", results.save_dir)

## Step 6 — ✅ Test latest trained model

This script will grab the latest trained runs, uses its best.pt to validate its correct file and do a test on the model to see if detection is happening correctly, looking for accuracy, bounding boxes and efficiency

In [ ]:
#@title Test latest trained model

import glob
import os
from ultralytics import YOLO

# Path where YOLO saves runs
RUNS_DIR = "/content/YoloV12-Object-Detection-Project/runs/completed-training"

# Folder where you want to save validation results
RESULTS_DIR = "/content/YoloV12-Object-Detection-Project/runs/test-results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Find the most recent training run
latest_run = max(glob.glob(os.path.join(RUNS_DIR, "*")), key=os.path.getmtime)
weights_path = os.path.join(latest_run, "weights", "best.pt")

print(f"Validating latest run: {latest_run}")
print(f"Using weights: {weights_path}")

# Load the best model from that run
model = YOLO(weights_path)

# Run validation on the TEST split (change metrics to the same trained metrics used)
metrics = model.val(
    data="DATA_YAML",
    split="test",
    imgsz=640,
    batch=-1
)

# Build results filename based on run name (e.g., train2 -> train2_metrics.txt)
run_name = os.path.basename(latest_run)
save_path = os.path.join(RESULTS_DIR, f"{run_name}_metrics.txt")

# Save metrics to file
with open(save_path, "w") as f:
    f.write(f"Validation results for run: {run_name}\n")
    f.write(f"Weights: {weights_path}\n\n")
    for k, v in metrics.items():
        f.write(f"{k}: {v}\n")

print(f"Metrics saved to {save_path}")


## Step 7 — 🔮 Inference Testing For Images And Videos

After completeing a model test from step 6, we will move onto testing with new images and videos. Which can be sourced online anywhere. This way testing will be more accurate as its new data that the model hasn't been trained/tested on.

In [ ]:
#@title Create folder to store test images and videos

import os
from google.colab import files
import shutil

base_dir = "/content/YoloV12-Object-Detection-Project/testing"
folder_name = "koala" #change to kangaroo if doing kangaroo dataset
target_path = os.path.join(base_dir, folder_name)

os.makedirs(target_path, exist_ok=True)
print(f"Folder created at: {target_path}")

# Prompt user to upload files
print("Please upload your image or video files:")
uploaded = files.upload()

# Move uploaded files into the target folder
for filename in uploaded.keys():
    shutil.move(filename, os.path.join(target_path, filename))
    print(f"Moved '{filename}' to '{target_path}'.")

print("All files uploaded and saved in your custom directory!")

In [ ]:
#@title Inference Testing with Images & Videos
import glob
import os
from ultralytics import YOLO

# Path where YOLO saves runs
RUNS_DIR = "/content/YoloV12-Object-Detection-Project/runs/completed-training"

# Path to your test folder (contains both images & videos)
TEST_DIR = "/content/YoloV12-Object-Detection-Project/testing/koala"

# Base path for inference results
OUTPUT_BASE = os.path.join(TEST_DIR, "inference_results")
os.makedirs(OUTPUT_BASE, exist_ok=True)

# Find the next available inference folder (inference1, inference2, ...)
i = 1
while os.path.exists(os.path.join(OUTPUT_BASE, f"inference{i}")):
    i += 1
OUTPUT_DIR = os.path.join(OUTPUT_BASE, f"inference{i}")

# Find the most recent training run
latest_run = max(glob.glob(os.path.join(RUNS_DIR, "*")), key=os.path.getmtime)
weights_path = os.path.join(latest_run, "weights", "best.pt")

print(f"Running inference with weights: {weights_path}")
print(f"Testing folder: {TEST_DIR}")
print(f"Results will be saved to: {OUTPUT_DIR}")

# Load model
model = YOLO(weights_path)

# Run inference (images + videos in same folder)
results = model.predict(
    source=TEST_DIR,   # folder containing both images & videos
    imgsz=640,
    conf=0.25,
    save=True,
    project=OUTPUT_DIR,
    name="",           # ensures results are saved directly in OUTPUT_DIR
    exist_ok=True
)

print(f"Inference complete. Results saved to: {OUTPUT_DIR}")



# Step 8 - 🪣 Choosing Final Model for Deployment

This step will go through every single trained model/validation results, and finds/selects the best model from the metrics provided. This way with the Final Model selected will go through final inference testing then Step 9 to be exported and used in real world deployment.

In [ ]:
#@title Find the best final model to use from test results (saves as a csv)

import glob
import os
import pandas as pd
from ultralytics import YOLO

# Paths
RUNS_DIR = "runs/train"
DATA_YAML = "DATA_YAML"  # <-- replace with your dataset yaml path
RESULTS_DIR = "/content/YoloV12-Object-Detection-Project/runs/model-results"
os.makedirs(RESULTS_DIR, exist_ok=True)  # make sure dir exists

# Output CSV path
CSV_PATH = os.path.join(RESULTS_DIR, "model_comparison.csv")

results_list = []

# Loop through all training runs
for run in sorted(glob.glob(os.path.join(RUNS_DIR, "*"))):
    weights_path = os.path.join(run, "weights", "best.pt")
    if not os.path.exists(weights_path):
        continue

    print(f"Validating {run} ...")
    model = YOLO(weights_path)
    metrics = model.val(data=DATA_YAML, split="test", imgsz=640, batch=-1)

    results_list.append({
        "run": os.path.basename(run),
        "weights": weights_path,
        "mAP50": metrics.box.map50,
        "mAP50-95": metrics.box.map,
        "precision": metrics.box.mp,
        "recall": metrics.box.mr
    })

# Save results to CSV
df = pd.DataFrame(results_list)
df.to_csv(CSV_PATH, index=False)

print(f"\nModel comparison saved to {CSV_PATH}")
print(df)



In [ ]:
#@title Compares Model Metrics in CSV and chooses the best model

import os
import shutil
import pandas as pd

# ==== CONFIG ====
CSV_PATH = "/content/YoloV12-Object-Detection-Project/runs/model-results/model_comparison.csv"
MODELS_DIR = "/content/YoloV12-Object-Detection-Project/models"
SUMMARY_DIR = "/content/YoloV12-Object-Detection-Project/model-resuslts"

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)

# loads csv file
df = pd.read_csv(CSV_PATH)

# checks csv columns (correct data)
required_cols = {"run", "weights", "mAP50-95", "mAP50", "precision", "recall"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"CSV missing required columns: {missing}")

for c in ["mAP50-95", "mAP50", "precision", "recall"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# picks best model from csv data metrics
best_row = df.sort_values(
    by=["mAP50-95", "mAP50", "precision"],  # tie-break if needed
    ascending=[False, False, False]
).iloc[0]

run = best_row["run"]
src = best_row["weights"]

if not isinstance(src, str) or not os.path.exists(src):
    raise FileNotFoundError(f"best.pt not found: {src}")

# Destination name: trainxxx_best.pt
base_name = f"{run}_best.pt"
dst = os.path.join(MODELS_DIR, base_name)

print(f"   Best model found: {run}")
print(f"   mAP50-95: {best_row['mAP50-95']:.4f} | mAP50: {best_row['mAP50']:.4f} | precision: {best_row['precision']:.4f}")
print(f"   Copying: {src} -> {dst}")

shutil.copy2(src, dst)

# Output and saving summary selection
summary_csv = os.path.join(SUMMARY_DIR, "best_model_summary.csv")
best_row.to_frame().T.to_csv(summary_csv, index=False)
print(f"\n Summary written to: {summary_csv}")



## Step 9 — ♟️ Exporting

Export the full respository incase you run into errors, you think the runtime is about to expire or you just want to save it to run later on.

In [ ]:
#@title Export Repository as Zip File
import shutil

shutil.make_archive('YoloV12-Object-Detection-Project', 'zip', '/content/YoloV12-Object-Detection-Project')

from google.colab import files
files.download('YoloV12-Object-Detection-Project.zip')

In [ ]:
## This will become a script to export as a ready model file

## Troubleshooting


- **Weights YAML missing:** ensure `models/yolo12s.pt` and `yaml/data.yaml` exist in the repo or update paths.
- **Val fails:** Train first; then rerun the validate cell.
- **Poor metrics:** Add more data, correct labels, tune `imgsz`/`batch`/`epochs`.
